In [1]:
import pathlib as pl

import glob
import pandas as pd
import numpy as np

import itertools, warnings

from collections import Counter, defaultdict
from typing import List, Dict, Tuple

## 1. Data Loading: Load PMI difference values

In [2]:
from utils import GROUP_PAIRED_WORDLIST, FEMALE_WORDS, MALE_WORDS, get_pmi_diff, get_gender_pairs_matrix

In [3]:
BASE_DIR = "../data"

# loads the PMI information precomputed based on the PILE co-occurrence counts
GENDER_PMI = pd.read_csv(f"{BASE_DIR}/pmi_by_gendered_expressions.csv", index_col=0)
print(len(GENDER_PMI))
GENDER_PMI.describe()

152515


,pmi__her,pmi__his,pmi__him,pmi__hers,pmi__mother,pmi__father,pmi__mom,pmi__dad,pmi__mummy,pmi__daddy,...,pmi__queen,pmi__king,pmi__queens,pmi__kings,pmi__princess,pmi__prince,pmi__princesses,pmi__princes,pmi__he,pmi__she
count,80439.000000,98771.000000,65608.000000,7537.000000,30706.000000,29684.000000,10998.000000,10495.000000,1717.000000,2977.000000,...,10119.000000,19446.000000,3313.000000,6617.000000,5412.000000,8203.000000,1266.000000,3825.000000,100828.000000,66891.000000
mean,-24.827642,-24.861843,-24.803681,-24.205257,-24.915487,-24.912725,-25.195694,-25.311220,-24.439117,-25.298108,...,-25.361834,-25.395301,-24.835216,-24.729553,-25.019586,-25.328138,-23.698000,-23.932025,-25.416424,-25.262707
std,1.563299,1.580690,1.506580,1.445382,1.324532,1.342077,1.403486,1.310585,1.538892,1.450787,...,1.435780,1.500407,1.723579,1.696653,1.477100,1.462137,1.718849,1.715749,1.514534,1.499545
min,-33.499275,-33.331670,-33.613325,-30.640672,-30.936595,-30.878193,-30.376505,-30.894450,-29.312282,-30.237593,...,-30.485745,-31.237119,-28.636829,-29.793686,-30.302262,-31.337065,-30.067307,-29.995963,-32.885207,-33.474327
25%,-25.685422,-25.682861,-25.638325,-25.184315,-25.755291,-25.742207,-26.175983,-26.204275,-25.419204,-26.245775,...,-26.331394,-26.407824,-25.942235,-25.915272,-26.010199,-26.314027,-24.823475,-25.134534,-26.219694,-26.079852
50%,-24.622905,-24.552552,-24.605622,-24.219955,-24.857188,-24.823301,-25.178166,-25.242244,-24.648881,-25.315466,...,-25.429962,-25.440690,-25.287083,-24.966771,-25.088823,-25.347642,-24.085664,-24.178638,-25.137871,-25.078310
75%,-23.743398,-23.767377,-23.746094,-23.240134,-24.040494,-24.000929,-24.242664,-24.399705,-23.807955,-24.462129,...,-24.524708,-24.469532,-24.229567,-23.792893,-24.122755,-24.380551,-22.893234,-22.914827,-24.351869,-24.212292
max,-20.458226,-20.727246,-19.520618,-18.544282,-18.927156,-19.795987,-17.653614,-18.349328,-16.823430,-15.609870,...,-17.835727,-18.720616,-16.770424,-17.806201,-17.986234,-18.367298,-15.972031,-16.816626,-20.899774,-21.228257


In [4]:
# Since we may want to perform some correlation with other gendered words
# we also define the PMI diff between words and other gendered word pairs
GENDER_PAIRS, GENDER_PAIRS_NUM_WORDS = get_gender_pairs_matrix(GENDER_PMI, GROUP_PAIRED_WORDLIST)
# ----------------------------------------------------------------------------
# compute PMI diff used in the main paper
# ----------------------------------------------------------------------------
# Most analysis will focus on the pmi_diff(she, he)
PMI_DIFF = get_pmi_diff(GENDER_PMI, "she", "he").sort_values("pmi(she)-pmi(he)")
# rename pmi difference column to be something less verbose :b
PMI_DIFF = PMI_DIFF.rename({"pmi(she)-pmi(he)": "pmi_diff"}, axis=1)
PMI_DIFF.sample(15, random_state=81273)

('she', 'he') pmi-defined words: 65912
('her', 'his') pmi-defined words: 75032
('her', 'him') pmi-defined words: 62131
('hers', 'his') pmi-defined words: 7536
! Pair (grandmother, grandfather) doesn't exist...
! Pair (grandma, grandpa) doesn't exist...
! Pair (stepmother, stepfather) doesn't exist...
! Pair (stepmom, stepdad) doesn't exist...
('mother', 'father') pmi-defined words: 26121
('mom', 'dad') pmi-defined words: 9150
('aunt', 'uncle') pmi-defined words: 5380
! Pair (aunts, uncles) doesn't exist...
('mummy', 'daddy') pmi-defined words: 1255
('sister', 'brother') pmi-defined words: 15727
('sisters', 'brothers') pmi-defined words: 8049
('daughter', 'son') pmi-defined words: 18721
('daughters', 'sons') pmi-defined words: 7276
('female', 'male') pmi-defined words: 28115
! Pair (females, males) doesn't exist...
! Pair (feminine, masculine) doesn't exist...
('woman', 'man') pmi-defined words: 31857
('women', 'men') pmi-defined words: 38861
! Pair (madam, sir) doesn't exist...
! Pair 

,word,pmi__she,pmi__he,pmi_diff
149350,worldviews,-26.607044,-25.767380,-0.839664
97697,overspend,-25.071416,-24.994815,-0.076601
26858,caricaturing,-24.687485,-24.163941,-0.523544
123998,slatted,-25.064006,-25.430709,0.366702
16674,attentions,-23.186299,-23.783683,0.597384
110979,rearward,-27.267943,-26.689328,-0.578615
2268,12153,-24.776008,-24.636996,-0.139012
25189,burping,-23.977679,-24.346041,0.368362
116091,roadblocks,-25.271944,-24.999363,-0.272581
124507,smellin,-23.624592,-24.394020,0.769428


## 2. Loading data - Load model scores for the different datasets

Say, PMI_DIFF(w, she, he), let us now compute the pmi of the words used for each of the benchmarks.

In [5]:
BASE_DIR = ".."

# list all the score files per dataset
DATASET_2_FILEPATHS = {
    "USE-5": glob.glob(f"{BASE_DIR}/results-words5/final-results/*__scores__*.csv"),
    # Baselines below ----
    "Winobias": glob.glob(f"{BASE_DIR}/results-baselines/final-results/*Winobias*__scores__*.csv"),
    "Winogender": glob.glob(f"{BASE_DIR}/results-baselines/final-results/*Winogender*__scores__*.csv"),
    # "StereoSet": glob.glob(f"../results-baselines/final-results/*StereoSet*__scores__*.csv"),
    # We define this ordering so that we can automatically obtain the same coloring scheme as
    # the one used for word analysis
    "USE-10": glob.glob(f"{BASE_DIR}/results-words10/final-results/*__scores__*.csv"),
    "USE-20": glob.glob(f"{BASE_DIR}/results-words20/final-results/*__scores__*.csv"),
}

DATASET_NAMES = list(DATASET_2_FILEPATHS.keys())
print(" Dataset names:\n  ->", DATASET_NAMES, "\n", "-" * 62)

# ------------------------------------------------------------------------------
#                                     Validation
# ------------------------------------------------------------------------------
# All datasets must have exact same number of files and ordered in the same way.
for dataset1, dataset2 in itertools.product(DATASET_NAMES, DATASET_NAMES):
    fps1 = [fp.rpartition("__scores__")[-1] for fp in DATASET_2_FILEPATHS[dataset1]] 
    fps2 = [fp.rpartition("__scores__")[-1] for fp in DATASET_2_FILEPATHS[dataset2]] 
    c1, c2 = Counter(fps1), Counter(fps2)
    assert len(c1 & c2) == len(c1), f"Validation failed for datasets: ({dataset1}, {dataset2})"

# !! Assumption: When scoring there was no change in the ordering of the templates and therefore
# every time we load the filepaths, we will have exactly the same ordering for all files (regardless
# of the scoring model).
DATASET_2_FILEPATHS = {k: sorted(v) for k, v in DATASET_2_FILEPATHS.items()}

print(" Number of files per dataset", "\n", "-" * 65)
for name, files in DATASET_2_FILEPATHS.items():
    print(" -> ", name, len(files))

 Dataset names:
  -> ['USE-5', 'Winobias', 'Winogender', 'USE-10', 'USE-20'] 
 --------------------------------------------------------------
 Number of files per dataset 
 -----------------------------------------------------------------
 ->  USE-5 28
 ->  Winobias 56
 ->  Winogender 28
 ->  USE-10 28
 ->  USE-20 28


#### Preprocess the datasets

Transform the datasets into the canonic form:

1. Transform model name into its canonic form: Extract from filepath name and add it as a column to the dataset.
3. Obtain information about model size: 
2. Obtain information about the interventions: Is the model trained on duplicated data (is_deduped=False) or non-duplicated data (is_deduped=True).
3. Obtain information about whether the test sentence pair is natural (is_natural=True) or whether is unnatural for one of the variants in the pair (is_natural=False)
4. Obtain information about the model family.

In [6]:
from model_utils import *

# Mapping from dataset name to the file dataframes
DATASET_2_FILES = defaultdict(list)

# Read each individual filepath, creating an association <str, list<dataframe>>.
# every str should have a list of the same size.

# To test the impact of ommiting the unnaturalness check, CHANGE THE VALUE BELOW TO FALSE
FILTER_UNNATURAL = True
# ------------------------------ ------------------------------ ------------------------------
DATASET_2_FILES = {
    dataset: [read_filepath(fp, dataset, filter_unnatural=FILTER_UNNATURAL) for fp in sorted(fps)]
    for dataset, fps in DATASET_2_FILEPATHS.items()
}

# Merge all the dataframes into a single big dataframe that contains the information of all models
# for each dataset. We've created a original index to keep track of the unique sentences.
# Sort the files per (model, orig_index)
DATASET_2_FILES = {
    dataset: pd.concat(dfs).sort_values(["model", "orig_index"]).reset_index(drop=True)
    for dataset, dfs in DATASET_2_FILES.items()
}

# Number of models being evaluated 
NUM_EVAL_MODELS = []
MODELS = []
for dataset, df  in DATASET_2_FILES.items():
    print(dataset, df["model"].nunique())
    MODELS.extend(df["model"].unique())
    NUM_EVAL_MODELS.append(df["model"].nunique())
    
# We force the number of models to be the same across all datasets
assert len(set(NUM_EVAL_MODELS)) == 1, \
    f"Found various model sizes: {NUM_EVAL_MODELS}"

NUM_EVAL_MODELS = NUM_EVAL_MODELS[0]
print("Evaluating", NUM_EVAL_MODELS, "models:")
MODELS = list(sorted(set(MODELS)))
print(" -", "\n - ".join(MODELS))

Filtered -549 unnatural, removed from USE-5
Number of unique 'likely_under' labels (should be 1): 2
Filtered -549 unnatural, removed from USE-5
Number of unique 'likely_under' labels (should be 1): 2
Filtered -549 unnatural, removed from USE-5
Number of unique 'likely_under' labels (should be 1): 2
Filtered -549 unnatural, removed from USE-5
Number of unique 'likely_under' labels (should be 1): 2
Filtered -549 unnatural, removed from USE-5
Number of unique 'likely_under' labels (should be 1): 2
Filtered -549 unnatural, removed from USE-5
Number of unique 'likely_under' labels (should be 1): 2
Filtered -549 unnatural, removed from USE-5
Number of unique 'likely_under' labels (should be 1): 2
Filtered -549 unnatural, removed from USE-5
Number of unique 'likely_under' labels (should be 1): 2
Filtered -549 unnatural, removed from USE-5
Number of unique 'likely_under' labels (should be 1): 2
Filtered -549 unnatural, removed from USE-5
Number of unique 'likely_under' labels (should be 1): 2


## Post processing: 

In this section, we will carry some processing of the templates (column "template").


**1. Remove placeholders from templates** : We first remove the placeholders (e.g., "{PRONOUN}", "{PRONOUN1}", "{PRONOUN2}", "{PRONOUN2}self") from the template.

**2. Remove stopwords from the templates**: We use **spacy**'s stopwords except that we add back some of the pronouns, effectively following the approach in [Razeghi et al 2022](https://aclanthology.org/2022.emnlp-demos.39/).

**3. Parse each template**: We use **spacy** tokenizer since this was what was used by [Razeghi et al 2022](https://aclanthology.org/2022.emnlp-demos.39/). While NTLK is much faster, it doesn't group together words like "self-care", which is treated as single word by spacy tokenizer. Therefore, we've updated the script to consider the spacy tokenization. Applying it to the whole DATASET_2_FILES[dataset] will be too time-consuming, so we will apply to the first portion of the data and then concatenate it to the dataframe.


### Filtering


Before applying the processing, we will first obtain the top unique templates by focusing on the subset of data of the first listed model.

In [7]:
# Validation (!sanity check)
# When selecting a data slice from the big dataframe
# we must guarantee that the sentences match to one another
# (that is necessary because the remaining of the code is relying
# on ordering of the dataframes)
def check_slices(dataset: pd.DataFrame, data2files: dict, models: List[str]):
    """Check for the ordering of the rows in ``dataset`` correspond to the
    ones in ``data2files``. Since the data2files are ordered by models,
    we will focus on that."""
    slices = []
    for model in models:
        df = data2files[dataset]
        df = df[df["model"] == model].copy()
        if len(slices) > 1:
            assert np.array_equal(slices[-1]["template"].values, df["template"].values)    
        slices.append(df)
        
    
for dataset in DATASET_NAMES:
    print("Checking slices for dataset:", dataset)
    check_slices(dataset=dataset, data2files=DATASET_2_FILES, models=MODELS)
    
# -----------------------------------------------------------------------------
# ^Note: if the check above does not throw an error, then it means that the
# templates can stack up based on the model, so it's ok to apply the processing
# to the first model and then create NUM_EVAL_MODEL copies of that and insert
# in the dataframe!!
# -----------------------------------------------------------------------------
DATASET_2_TEMPLATES = {
    dataset: df[df["model"] == MODELS[0]]["template"].values.tolist()
    for dataset, df in DATASET_2_FILES.items()
}

Checking slices for dataset: USE-5
Checking slices for dataset: Winobias
Checking slices for dataset: Winogender
Checking slices for dataset: USE-10
Checking slices for dataset: USE-20


### Processing (using Spacy tokenizer)

In [8]:
try:
    import spacy
except:
    !pip install spacy
    !python -m spacy download en_core_web_md
import spacy
import nltk 
import re, string


from nltk.corpus import stopwords
nltk.download('stopwords')


PRONOUNS = ["she", "her", "hers", "he", "his", "him", "himself", "herself"]
SPACY_PARSER = spacy.load("en_core_web_md", disable=["ner", "tagger"])


def postprocess_spacy(templates, pronouns=PRONOUNS):
    def word_tokenize(sentence: str, pronouns: list, remove_stopwords: bool=True, remove_punct: bool=True):
        doc = SPACY_PARSER(sentence)
        # Extract the tokens that are not stopwords
        tokens = [token.text for token in doc 
                  if (token.text in pronouns) or (not token.is_stop and not token.is_punct)]
        return [t for t in tokens if len(t.strip()) > 0]

    templates = [t.lower() for t in templates]
    # Step 1. Remove placeholders from the templates
    templates = [t.replace("{pronoun2}self", "") for t in templates]
    templates = [re.sub(r"\{pronoun([0-2]{1})?\}", "", t) for t in templates]
    # Step 2. Parse the sentence
    templates = [word_tokenize(t, pronouns) for t in templates]
    return templates


def postprocess_nltk(templates, pronouns=PRONOUNS):
    from nltk.tokenize import word_tokenize

    nltk_stopwords = set(stopwords.words('english'))
    # We know that some sentences have some other references to other entities,
    # let's keep some pronouns
    nltk_stopwords -= set(pronouns)
    punct = string.punctuation
    
    templates = [t.lower() for t in templates]
    # Remove pronouns first
    templates = [t.replace("{pronoun2}self", "") for t in templates]
    templates = [re.sub(r"\{pronoun([0-2]{1})?\}", "", t) for t in templates]
    
    # Remove stopwords and punct
    templates = [[w for w in word_tokenize(t) if w not in punct and w not in nltk_stopwords] for t in templates]
    return templates


DATASET_2_CANONIC_TEMPLATES_SPACY = {}
DATASET_2_CANONIC_TEMPLATES_NLTK = {}

for dataset, templates in DATASET_2_TEMPLATES.items():
    DATASET_2_CANONIC_TEMPLATES_SPACY[dataset] = postprocess_spacy(templates)
    DATASET_2_CANONIC_TEMPLATES_NLTK[dataset] = postprocess_nltk(templates)

[nltk_data] Downloading package stopwords to /home/cbelem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/cbelem/projects/tokenization-proj/venv/lib/python3.8/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


## Determine gender co-occurrence for each word 

In this section, we iterate the templates and compute the gender co-occurrence values for each sentence in the benchmarks. Optionally, you can weight the values of each word by the likelihood of being toxic or having negative sentiment. If such values are not provided, we assume each word is worth the same value of 1 unit.


In [9]:
## Convert dataframe to mapping from word to pmi diff for easy access
WORD2PMI = PMI_DIFF[["word", "pmi_diff"]].set_index("word").to_dict()["pmi_diff"]
WORD2WEIGHTS = defaultdict(lambda: 1)

## ----------------------------------------------------------------
## Weighting words based on frequency
## ----------------------------------------------------------------
FREQ_WORDS = pd.read_csv("../data/pmi_file_w_counts.csv", index_col=0)
FREQ_WORDS["log_freq"] = np.log(FREQ_WORDS["freq"])

## uncomment one of the lines below if you prefer weighting each word based
## on the frequency of each individual word
# WORD2WEIGHTS = FREQ_WORDS[["word", "freq"]].set_index("word").to_dict()["freq"]
# WORD2WEIGHTS = FREQ_WORDS[["word", "log_freq"]].set_index("word").to_dict()["log_freq"]

## ----------------------------------------------------------------
## Weighting words based on toxicity/sentiment
## ----------------------------------------------------------------
## TODO:
## -> Define toxicity for each word
## -> Define sentiment polarity for each word (?)
## Define a 1-to-1 mapping and assign the variable WORD2WEIGHTS

In [10]:
def compute_pmi_diff_per_sentences(
        templates: List[List[str]],
        word2pmi: dict,
        word2weights: dict,
    ) -> List[List[float]]:
    """Computes the PMI difference per individual token in the provided sentences.
    
    Notes
    -----
    It assumes the templates/sentences are already provided as a list of tokens.
    It returns two lists: the first one contains the list of pmi values for each of
    the provided words (some tokens won't have a PMI value associated); the second
    list contains the 1-1 mapping from word to pmi value and their weights.
    """
    pmi_values = []
    words_with_pmi = []
    
    for template in templates:
        pmi = np.array([word2weights[w] * word2pmi.get(w) for w in template if word2pmi.get(w) is not None])
        pmiwords = [{
            "word": w, 
            "pmi": round(word2pmi.get(w), 2),
            "weight": round(word2weights[w], 2),
        } for w in template if word2pmi.get(w) is not None]
        
        pmi_values.append(pmi)
        words_with_pmi.append(pmiwords)
            
    return pmi_values, words_with_pmi
        

PMI_PER_SENTENCES_NLTK = {dataset: 
                          compute_pmi_diff_per_sentences(templates, WORD2PMI, WORD2WEIGHTS)
                          for dataset, templates in DATASET_2_CANONIC_TEMPLATES_NLTK.items()
}

PMI_PER_SENTENCES_SPACY = {dataset: 
                          compute_pmi_diff_per_sentences(templates, WORD2PMI, WORD2WEIGHTS)
                          for dataset, templates in DATASET_2_CANONIC_TEMPLATES_SPACY.items()
}

Since in general **spacy** tokenizer leads to higher pct of examples being matched with a word. We will use the **spacy** tokenized templates to conduct the analysis (it increases the coverage of the constraints).

In [11]:
PMI_PER_TEMPLATES = {}
PMIWORDS_PER_TEMPLATES = {}

# Change the PMI_PER_SENTENCES_SPACY with PMI_PER_SENTENCES_NLTK
# to use NLTK tokenization instead.
# for dataset, pmi_per_sents_values in PMI_PER_SENTENCES_NLTK.items():
for dataset, pmi_per_sents_values in PMI_PER_SENTENCES_SPACY.items():
    pmi_vals, words_per_pmi = pmi_per_sents_values
    
    PMI_PER_TEMPLATES[dataset] = pmi_vals
    PMIWORDS_PER_TEMPLATES[dataset] = words_per_pmi

### Compute the constraint: MaxPMI(s)

In this section, we compute the max gender PMI value per sentence. This consists of determining that's the max absolute word-level PMI value.

In [12]:
MAXGENDER_COL = "max_gender_pmi"

def max_gender_pmi(templates_pmi: List[List[str]], col: str) -> List[dict]:
    """Compute the maximum PMI diff per sentence."""
    def _max_pmi(lst_pmis: List[str]) -> float:
        if len(lst_pmis) > 0:
            idx = np.argmax(np.abs(lst_pmis))
            return lst_pmis[idx]
    
    results = []
    for template_pmi in templates_pmi:
        max_val = _max_pmi(template_pmi)
        results.append({col: max_val, f"{col}_invalid": max_val is None, "template_words_pmi": template_pmi})
        
    return results

In [13]:
# Contains the max gender pmi values per sentence
MAX_GENDER_PMI = {dataset: max_gender_pmi(templates_pmi, MAXGENDER_COL) 
                  for dataset, templates_pmi in PMI_PER_TEMPLATES.items()}

MAX_GENDER_PMI_LONG = []
for dataset, lst_value_dicts in MAX_GENDER_PMI.items():
    for value_dict in lst_value_dicts:
        r = {k: v for k, v in value_dict.items()}
        r["dataset"] = dataset
        MAX_GENDER_PMI_LONG.append(r)

MAX_GENDER_PMI_LONG = pd.DataFrame(MAX_GENDER_PMI_LONG)
        
# Adds the information to the original dataset with all models
# originally, preserved in the variable DATASET_2_FILES
DATASET_W_CONSTRAINTS = {dataset: pd.DataFrame(values * NUM_EVAL_MODELS)
                  for dataset, values in MAX_GENDER_PMI.items()}

# ------------------------------------------------------------------------------
# 
#                        Dataset w/ MaxGender PMI constraint!
# 
# ------------------------------------------------------------------------------
DATASET_W_CONSTRAINTS = {
    dataset: pd.concat((DATASET_2_FILES[dataset], DATASET_W_CONSTRAINTS[dataset]), copy=True, axis=1)
    for dataset in DATASET_NAMES
}

DATASET_W_CONSTRAINTS[DATASET_NAMES[0]].columns

Index(['orig_index', 'word', 'target_word', 'sentence', 'has_placeholder',
       'template', 'modifications', 'likely_under', 'is_natural', 'has_word',
       'is_revised', 'M_num_tokens', 'M_logprob', 'M_template', 'F_num_tokens',
       'F_logprob', 'F_template', 'FM_logprob', 'model', 'dataset',
       'is_deduped', 'is_intervention', 'orig_model_name', 'model_size',
       'model_family', 'max_gender_pmi', 'max_gender_pmi_invalid',
       'template_words_pmi'],
      dtype='object')

## Persist post-processed results



In [ ]:
for dataset, df in DATASET_W_CONSTRAINTS.items():
    df.to_csv(f"../results/{dataset}-no-maxpmi-constraint.csv.gz", index=None)